In [1]:
import numpy as np
import pandas as pd
import random
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
import datetime
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from tqdm import tqdm

X_train = np.loadtxt("./data/X_train_-999to-5_removal.txt")
y_train = np.loadtxt("./data/y_train_-999to-5_removal.txt")
X_valid = np.loadtxt("./data/X_valid_-999to-5_removal.txt")
y_valid = np.loadtxt("./data/y_valid_-999to-5_removal.txt")
X_test = np.loadtxt("./data/X_test_-999to-5_removal.txt")
y_test = np.loadtxt("./data/y_test_-999to-5_removal.txt")

In [2]:
X_train = np.loadtxt("./data/X_train_-999to-5_removal.txt")
y_train = np.loadtxt("./data/y_train_-999to-5_removal.txt")
X_valid = np.loadtxt("./data/X_valid_-999to-5_removal.txt")
y_valid = np.loadtxt("./data/y_valid_-999to-5_removal.txt")
X_test = np.loadtxt("./data/X_test_-999to-5_removal.txt")
y_test = np.loadtxt("./data/y_test_-999to-5_removal.txt")

In [3]:
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(16437, 20)
(16437,)
(2055, 20)
(2055,)
(2055, 20)
(2055,)


(None, None)

In [4]:
class SHAPEDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        x = torch.tensor(self.x[index], dtype=torch.float32)
        y = torch.tensor(self.y[index], dtype=torch.float32)
        return x, y

    def __len__(self):
        return len(self.x)

In [5]:
# project root path
project_path = "./cnn/"
# define log directory
# must be a subdirectory of the directory specified when starting the web application
# it is recommended to use the date time as the subdirectory name
log_dir = project_path + "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = project_path + "shape_cnn_model_e300_b512_lr001_2conv1d_-999to-5_head_tail_removal.pt"

# the device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device



## Creating our Model
---

In [6]:
# build the CNN model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # the first convolution layer, output shape (batch_size, 4, 20)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding='same')
        # the first pooling layer, max pooling, pooling size=3 , stride=2, output shape (batch_size, 4, 10)
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        # the second convolution layer, output shape (batch_size, 8, 10)
        self.conv2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding='same')
        # the second pooling layer, max pooling, pooling size=3 , stride=2, output shape (batch_size, 8, 5)
        self.pool2 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        # flatten layer, for the next fully connected layer, output shape (batch_size, 8*5)
        self.flatten = nn.Flatten()
        # fully connected layer, 4 nodes, output shape (batch_size, 4)
        self.fc1 = nn.Linear(8 * 5, 4)
        # Dropout layer, dropout rate = 0.2
        self.dropout = nn.Dropout(0.2)
        # fully connected layer, 1 node (prediction), output shape (batch_size, 1)
        self.fc2 = nn.Linear(4, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        # x.shape = (batch_size, 20)
        # reshape the tensor with shape (batch_size, 20) to (batch_size, 1, 20)
        x = x.reshape(-1, 1, 20)
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [7]:
# define the training function and validation function
def train_steps(loop, model, criterion, optimizer):
    train_loss = []
    model.train()
    for step_index, (X, y) in loop:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        pred = torch.flatten(pred)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss = loss.item()
        train_loss.append(loss)
        pred = pred.detach().cpu().numpy()
        y = y.detach().cpu().numpy()
        #print(step_index)
        loop.set_postfix(loss=loss)
    return {"loss": np.mean(train_loss)}

In [8]:
def valid_steps(loop, model, criterion):
    valid_loss = []
    model.eval()
    with torch.no_grad():
        for step_index, (X, y) in loop:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = criterion(pred, y).item()

            valid_loss.append(loss)
            pred = pred.detach().cpu().numpy()
            y = y.detach().cpu().numpy()
            #print(step_index)
            loop.set_postfix(loss=loss)
    return {"loss": np.mean(valid_loss)}

In [9]:
def train_epochs(train_dataloader, valid_dataloader, model, criterion, optimizer, config, writer):
    num_epochs = config['num_epochs']
    train_loss_ls = []
    valid_loss_ls = []
    min_valid_loss = np.inf
    for epoch in range(num_epochs):
        #print('length of train dataloader: ',str(len(train_dataloader)))
        train_loop = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        #print('length of validation dataloader: ',str(len(valid_dataloader)))
        valid_loop = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader))
        train_loop.set_description(f'Epoch [{epoch + 1}/{num_epochs}]')
        valid_loop.set_description(f'Epoch [{epoch + 1}/{num_epochs}]')
        #print('train step: ')
        train_metrix = train_steps(train_loop, model, criterion, optimizer)
        #print('validation step: ')
        valid_metrix = valid_steps(valid_loop, model, criterion)

        train_loss_ls.append(train_metrix['loss'])
        valid_loss_ls.append(valid_metrix['loss'])
        
        # only save the model when the validation drops
        if (valid_metrix['loss']< min_valid_loss):
            torch.save(model.state_dict(), model_path)
            print('model improved and saved\n')

        print(f'Epoch {epoch + 1}: '
              f'train loss: {train_metrix["loss"]} ')
        print(f'Epoch {epoch + 1}: '
              f'valid loss: {valid_metrix["loss"]} ')

        writer.add_scalar('train/loss', train_metrix['loss'], epoch) # every epoch
        writer.add_scalar('validation/loss', valid_metrix['loss'], epoch)

    return {'train_loss': train_loss_ls,
            'valid_loss': valid_loss_ls,}

In [10]:
def plot_history_torch(history,config):
    
    plt.figure(figsize=(8, 8))
    plt.plot(history['train_loss'])
    plt.plot(history['valid_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.savefig("b"+str(config['batch_size'])+'_e'+str(config['num_epochs'])+'_remove-999_loss.png')
    plt.show()

In [11]:
def main():
    config = {
        'seed': 40,  # the random seed
        'num_epochs': 300,
        'batch_size': 512,
        'lr': 0.001,
    }

    train_dataset = SHAPEDataset(X_train, y_train)
    valid_dataset = SHAPEDataset(X_valid, y_valid)
    test_dataset = SHAPEDataset(X_test, y_test)
    
    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

    # define the model
    model = Model()
    criterion = nn.MSELoss()
    
    # predict on test dataset with untrained model and note down the mse
    y_pred = []
    with torch.no_grad():
        for step_index, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            pred = pred.detach().cpu().numpy()
            y_pred.extend(pred)
        
        y_pred_before = np.concatenate(y_pred)
        y_pred_before_tensor = torch.Tensor(y_pred_before)
        y_test_tensor = torch.Tensor(y_test)
        y_test_mse_before = criterion(y_test_tensor, y_pred_before_tensor)

    #start real training
    if os.path.exists(model_path):
        # import the pre-trained model if it exists
        print('Import the pre-trained model, skip the training process')
        model.load_state_dict(torch.load(model_path))
        model.eval()
    else:
        # build the CNN model
        model = model.to(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'])

        # print the model structure
        # summary(model, (config['batch_size'], X_train.shape[1]), col_names=["input_size", "kernel_size", "output_size"],
               # verbose=2)

        # define the Tensorboard SummaryWriter
        writer = SummaryWriter(log_dir=log_dir)
        # train and evaluate model
        history = train_epochs(train_dataloader, valid_dataloader, model, criterion, optimizer, config, writer)
        writer.close()
        # save the model
        # torch.save(model.state_dict(), model_path)
        # plot the training history
        plot_history_torch(history,config)
        
    # write train loss and validation loss into pickle files
    dict_a = {'num_epochs':config['num_epochs'], 'batch_size':config['batch_size'], 'lr':config['lr'], 'loss': history['train_loss'], 'valid_loss': history['valid_loss']}
    pickle.dump(dict_a, open('./cnn/pickle_data/data_e'+str(config['num_epochs'])+ '_b'+str(config['batch_size'])+'_lr'+str(config['lr'])+'_2conv1d_remove_-999'+'.pkl', 'wb'))

    # predict on test dataset with trained model and note down the mse, compare with the untrained model results
    y_pred = []
    model.eval()
    with torch.no_grad():
        for step_index, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            pred = pred.detach().cpu().numpy()
            y_pred.extend(pred)
        
        y_pred = np.concatenate(y_pred)
        y_pred_tensor = torch.Tensor(y_pred)
        y_test_tensor = torch.Tensor(y_test)
        y_test_mse_after = criterion(y_test_tensor, y_pred_tensor)

        print('MSE loss for test dataset before optimizing: ', y_test_mse_before)
        print('MSE loss for test dataset after optimizing: ', y_test_mse_after)


In [ ]:
main()

Epoch [1/300]:   0%|                                                                  | 0/33 [00:00<?, ?it/s, loss=1.53]

0
1
2


Epoch [1/300]:   0%|                                                                  | 0/33 [00:00<?, ?it/s, loss=1.49]

3
4


Epoch [1/300]:  48%|███████████████████████████▋                             | 16/33 [00:00<00:00, 75.65it/s, loss=1.29]

...

Epoch [177/300]:  15%|████████▎                                              | 5/33 [00:00<00:00, 48.27it/s, loss=0.867]

5


Epoch [177/300]:  15%|████████▎                                              | 5/33 [00:00<00:00, 48.27it/s, loss=0.865]

6
7
8


Epoch [177/300]:  15%|████████▎                                              | 5/33 [00:00<00:00, 48.27it/s, loss=0.795]